In [1]:
import matplotlib
matplotlib.use('TkAgg')
import ecg_noise_detector.src.ecg_noise_detector.noiseDetector as nd

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [5]:
# Use this if changing noiseDetector.py
import importlib
importlib.reload(nd)

<module 'ecg_noise_detector.src.ecg_noise_detector.noiseDetector' from 'C:\\Users\\daniel\\Documents\\CambridgeSoftwareProjects\\ecg-signal-quality\\ecg_noise_detector\\src\\ecg_noise_detector\\noiseDetector.py'>

### Load the dataset

In [2]:
dataset = pd.read_pickle("CinC2017Data/database.pk")

In [3]:
dataset["length"] = dataset["data"].map(lambda arr: arr.shape[-1])
# select only the 30s length records
dataset = dataset[dataset["length"] == 9000]
dataset["data"] = dataset["data"].map(lambda d: d[0])

In [4]:
# dataset["onehot"] = dataset["class"].map(generate_onehot)
def generate_index(c):
    if c == "N":
        return 0
    if c == "O":
        return 0
    if c == "A":
        return 0
    if c == "~":
        return 1

dataset["class_index"] = dataset["class"].map(generate_index)

train_dataset, test_dataset = train_test_split(dataset, test_size=0.15, stratify=dataset["class_index"])

print(len(test_dataset.index))

897


In [5]:
print(dataset["class"].value_counts())

N    3695
O    1655
A     504
~     123
Name: class, dtype: int64


### An aside on optimising the length transform

In [19]:
lt_yoav = nd._length_transfrom(test_dataset["data"][0], 100)

In [20]:
lt_mine = nd._length_transform_faster(test_dataset["data"][0], 100)

In [53]:
def test_code():
    ecgs = test_dataset["data"].head(20)
    nd.is_noisy_batch(ecgs, fs=300)

import cProfile
cProfile.run('test_code()')

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\daniel\Documents\CambridgeSoftwareProjects\ecg-signal-quality\ecg_noise_detector\src\ecg_noise_detector\noiseDetector.py:81: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  p = pd.Series(index=ecgs.index)


Processing ecg: A04315
Processing ecg: A03779
Processing ecg: A05397
Processing ecg: A03823
Processing ecg: A00227
Processing ecg: A08170
Processing ecg: A03236
Processing ecg: A07489
Processing ecg: A04197
Processing ecg: A00860
Processing ecg: A04877
Processing ecg: A05200
Processing ecg: A01613
Processing ecg: A02253
Processing ecg: A05897
Processing ecg: A05041
Processing ecg: A07776
Processing ecg: A06002
Processing ecg: A07440
Processing ecg: A01792
         5192772 function calls (5113124 primitive calls) in 4.064 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.064    4.064 3875118959.py:1(test_code)
     2840    0.001    0.000    0.012    0.000 <__array_function__ internals>:177(all)
     3360    0.004    0.000    0.021    0.000 <__array_function__ internals>:177(amax)
     8480    0.015    0.000    0.058    0.000 <__array_function__ internals>:177(any)
     1720    0.001    0.000   

### Run Yoav's classifier and analyse the results

In [7]:
ecgs = test_dataset["data"]
test_dataset["predictions"] = nd.is_noisy_batch(ecgs, fs=300, filter=False)

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\daniel\Documents\CambridgeSoftwareProjects\ecg-signal-quality\ecg_noise_detector\src\ecg_noise_detector\noiseDetector.py:81: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  p = pd.Series(index=ecgs.index)


Processing ecg: A00503
Processing ecg: A06949
Processing ecg: A05398
Processing ecg: A08103
Processing ecg: A05609
Processing ecg: A02815
Processing ecg: A07811
Processing ecg: A06506
Processing ecg: A06564
Processing ecg: A08068
Processing ecg: A08203
Processing ecg: A06871
Processing ecg: A00812
Processing ecg: A03623
Processing ecg: A00256
Processing ecg: A03910
Processing ecg: A03932
Processing ecg: A07280
Processing ecg: A07013
Processing ecg: A00022
Processing ecg: A02472
Processing ecg: A08055
Processing ecg: A07819
Processing ecg: A01137
Processing ecg: A07482
Processing ecg: A00415
Processing ecg: A04376
Processing ecg: A04510
Processing ecg: A03410
Processing ecg: A03156
Processing ecg: A07249
Processing ecg: A05010
Processing ecg: A08245
Processing ecg: A08237
Processing ecg: A03766
Processing ecg: A00189
Processing ecg: A00281
Processing ecg: A01817
Processing ecg: A00863
Processing ecg: A00908
Processing ecg: A03745
Processing ecg: A04434
Processing ecg: A03262
Processing 

In [9]:
test_dataset["predictions"] = test_dataset["predictions"].astype(int)

Plot examples of the data with window by window classification of the errors

In [29]:
i = 100

print(test_dataset.iloc[i]["class"])
nd.plot_ecg(test_dataset.iloc[i]["data"], fs=300)
plt.show()

O


C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


<Figure size 1169x827 with 3 Axes>

In [10]:
conf_mat = confusion_matrix(test_dataset["class_index"], test_dataset["predictions"])

def F1_ind(conf_mat, ind):
    return (2 * conf_mat[ind, ind])/(np.sum(conf_mat[ind]) + np.sum(conf_mat[:, ind]))

print(f"Normal F1: {F1_ind(conf_mat, 0)}")
print(f"Other F1: {F1_ind(conf_mat, 1)}")

ConfusionMatrixDisplay.from_predictions(test_dataset["class_index"], test_dataset["predictions"], display_labels=["sufficient quality", "insufficient quality"], cmap="inferno")
plt.show()

Normal F1: 0.556920556920557
Other F1: 0.055846422338568937


In [11]:
print(f"Sensitivity: {conf_mat[1, 1]/np.sum(conf_mat[1])}")
print(f"Specificity: {conf_mat[0, 0]/np.sum(conf_mat[0])}")

Sensitivity: 0.8888888888888888
Specificity: 0.38680318543799774


In [32]:
false_positives = test_dataset[(test_dataset["predictions"] == 1) & (test_dataset["class_index"] == 0)]
false_negatives = test_dataset[(test_dataset["predictions"] == 0) & (test_dataset["class_index"] == 1)]
noisy = test_dataset[(test_dataset["predictions"] == 1) & (test_dataset["class_index"] == 1)]

In [25]:
nd.plot_ecg(false_positives.iloc[50]["data"], fs=300)

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


<Figure size 1169x827 with 3 Axes>

In [30]:
nd.plot_ecg(false_negatives.iloc[0]["data"], fs=300)

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning:

Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



<Figure size 1169x827 with 3 Axes>

In [33]:
nd.plot_ecg(noisy.iloc[0]["data"], fs=300)

C:\Users\daniel\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning:

Trying to unpickle estimator SVC from version 1.0.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



<Figure size 1169x827 with 3 Axes>

## Try Training the SVM on the physionet data ourselves and see what the results are

In [37]:
# Extract features for 5s segments of the data using Yoav's code

ecg_features = []
i = 0

for ind, x in dataset.iterrows():
    print(f"Processing ecg {i}/{len(dataset.index)}\r", end="")
    df = nd._process_ecg(x["data"], fs=300, filter=False)
    df["class_index"] = x["class_index"]
    ecg_features.append(df)
    i += 1


df = pd.concat(ecg_features, keys=dataset.index)

In [ ]:
pk_path = "CinC2017Data/database_yeov_processed.pk"
df.to_pickle(pk_path)

In [10]:
df = pd.read_pickle("CinC2017Data/database_yeov_processed.pk")

In [11]:
# Normalise some features to have 0 mean and variance 1 (other elements are already limited/normalised)

var_1_features = ["sSQI", "kSQI"]
for f in var_1_features:
    df[f] = (df[f] - df[f].mean())/df[f].var()

In [12]:
# Train test split per ECG (not per 5s segment as they could overlap)

ind = np.unique(np.array(df.index.get_level_values(0)))
classes = np.array([df["class_index"].loc[i, 0] for i in ind])

from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
n_fold = 4
for i, (train_ind, test_ind) in enumerate(kf.split(ind)):
    if n_fold == i:
        train_dataset = df.loc[ind[train_ind]]
        test_dataset = df.loc[ind[test_ind]]

print(train_dataset.head())
print(test_dataset.head())

              sSQI      kSQI      pSQI    basSQI      bSQI      rSQI  \
A00001 0 -0.341007 -0.976092  0.473548  0.860109  0.666667  0.600000   
       1  0.393465 -0.750919  0.438357  0.881651  0.750000  0.571429   
       2  1.256622  1.859929  0.457808  0.973036  0.600000  0.625000   
       3  1.156859  1.376037  0.483638  1.026441  0.750000  0.571429   
       4  1.202952  1.498237  0.442891  1.009925  0.857143  0.538462   

          class_index  
A00001 0            0  
       1            0  
       2            0  
       3            0  
       4            0  
              sSQI      kSQI      pSQI    basSQI  bSQI      rSQI  class_index
A06936 0  1.069304  1.374057  0.479157  0.981022   0.5  0.666667            0
       1  1.114087  1.387644  0.438886  0.950741   0.6  0.625000            0
       2  1.218040  1.988814  0.454056  0.989571   0.5  0.666667            0
       3  0.950834  0.928937  0.456997  0.960214   0.5  0.666667            0
       4  0.935834  0.865227  0.5

In [13]:
# train the model

class_weights = 1/train_dataset["class_index"].value_counts()
class_weights /= np.sqrt(np.dot(class_weights, class_weights))

model = SVC(class_weight=class_weights.to_dict())
# See documentation for default values e.g. use rbf, regularising C = 1
features = ["sSQI", "kSQI", "pSQI", "basSQI", "bSQI", "rSQI"]

model = model.fit(train_dataset[features].values, train_dataset["class_index"].values)

KeyboardInterrupt: 

In [38]:
predictions = model.predict(test_dataset[features].values)

test_dataset["predictions"] = predictions
results_df = test_dataset.groupby(level=0).mean()
results_df["predictions"] = results_df["predictions"].round()

ConfusionMatrixDisplay.from_predictions(results_df["class_index"], results_df["predictions"], display_labels=["sufficint quality", "insufficient quality"], cmap="inferno")
plt.show()
conf_mat = confusion_matrix(results_df["class_index"], results_df["predictions"])

def F1_ind(conf_mat, ind):
    return (2 * conf_mat[ind, ind])/(np.sum(conf_mat[ind]) + np.sum(conf_mat[:, ind]))

print(f"Normal F1: {F1_ind(conf_mat, 0)}")
print(f"Other F1: {F1_ind(conf_mat, 1)}")

Normal F1: 0.8620185275475378
Other F1: 0.16519174041297935


In [39]:
print(f"Sensitivity: {conf_mat[1, 1]/np.sum(conf_mat[1])}")
print(f"Specificity: {conf_mat[0, 0]/np.sum(conf_mat[0])}")

Sensitivity: 0.7777777777777778
Specificity: 0.7627264883520276


### Try putting features for each section into the classifier as one input

In [14]:
# Construct test and train matrices

features = ["sSQI", "kSQI", "pSQI", "basSQI", "bSQI", "rSQI"]
train_matrix = []
train_targets = []
indexes = []

for i in set(train_dataset.index.get_level_values(0)):
    indexes.append(i)
    train_matrix.append(train_dataset.loc[i][features].values.flatten())
    train_targets.append(train_dataset.loc[(i, 0)]["class_index"])

train_matrix = np.array(train_matrix)
train_targets = np.array(train_targets)

print(train_matrix.shape)
print(train_targets.shape)


test_matrix=  []
test_targets = []
for i in set(test_dataset.index.get_level_values(0)):
    test_matrix.append(test_dataset.loc[i][features].values.flatten())
    test_targets.append(test_dataset.loc[(i, 0)]["class_index"])


test_matrix = np.array(test_matrix)
test_targets = np.array(test_targets)

print(test_matrix.shape)

(4782, 36)
(4782,)
(1195, 36)


In [15]:
# train the model
class_weights = 1/train_dataset["class_index"].value_counts()
class_weights /= np.sqrt(np.dot(class_weights, class_weights))
print(class_weights.to_dict())

model = SVC(class_weight=class_weights.to_dict())
# See documentation for default values e.g. use rbf, regularising C = 1
features = ["sSQI", "kSQI", "pSQI", "basSQI", "bSQI", "rSQI"]

model = model.fit(train_matrix, train_targets)

{0: 0.018527170837030867, 1: 0.9998283572397691}


In [16]:
predictions = model.predict(test_matrix)

ConfusionMatrixDisplay.from_predictions(test_targets, predictions, display_labels=["sufficient quality", "insufficient quality"], cmap="inferno")
plt.show()
conf_mat = confusion_matrix(test_targets, predictions)

def F1_ind(conf_mat, ind):
    return (2 * conf_mat[ind, ind])/(np.sum(conf_mat[ind]) + np.sum(conf_mat[:, ind]))

print(f"Normal F1: {F1_ind(conf_mat, 0)}")
print(f"Other F1: {F1_ind(conf_mat, 1)}")

Normal F1: 0.8140601120733572
Other F1: 0.1451990632318501


In [17]:
print(f"Sensitivity: {conf_mat[1, 1]/np.sum(conf_mat[1])}")
print(f"Specificity: {conf_mat[0, 0]/np.sum(conf_mat[0])}")

Sensitivity: 0.8611111111111112
Specificity: 0.6893874029335634


In [30]:
false_positives = np.array(list(set(test_dataset.index.get_level_values(0))))[(test_targets == 0) * (predictions == 1)]
print(false_positives[0])

print(test_matrix[(test_targets == 0) * (predictions == 1)][1])

plt.plot(dataset["data"].loc[false_positives[1]])
plt.show()

A07546
[-0.42255649 -1.55240177  0.4495139   0.81001201  0.5         0.66666667
 -0.87647898 -1.74441361  0.31188967  0.86828259  0.57142857  0.63636364
 -0.72636483 -2.10796461  0.39596124  0.90054756  0.4375      0.69565217
 -0.82719949 -1.62955888  0.2995643   0.98486546  0.58333333  0.63157895
 -0.74217845 -1.59165107  0.33631894  0.94724882  0.53846154  0.65
 -0.50393047 -1.45371067  0.31330429  0.84007719  0.63636364  0.61111111]


Why am I having poorer accuracy here?
 - Is it because the SVM is not able to understand so many variables